In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load in Data from GitHub
data = pd.read_csv('final_data.csv')
data.head()

,title,audience_score,tomato_meter,rating,rating_contents,director,writer,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,wiki_page,plot,genre,language
0,the in-laws,83.0,88.0,NaN,NaN,arthur hiller,andrew bergman,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979,"peter falk, alan arkin",https://en.wikipedia.org/wiki/the_in-laws_(197...,the daughter of mild-mannered manhattan dentis...,comedy,english
1,race the sun,41.0,22.0,NaN,NaN,charles t. kanganis,NaN,1700000.0,NaN,surround,NaN,0.0,55731.0,100,1996,"halle berry, james belushi",https://en.wikipedia.org/wiki/race_the_sun,"a new science teacher, miss sandra beecher, (h...",drama,english
2,paul blart: mall cop,43.0,34.0,pg,"mild crude humor, language, some violence",steve carr,kevin james,183000000.0,"sony pictures releasing,",NaN,NaN,26000000.0,NaN,91,2009,"jayma mays, keir o'donnell, bobby cannavale, s...",https://en.wikipedia.org/wiki/paul_blart:_mall...,"paul blart lives in west orange, new jersey wi...",comedy,english
3,not without my daughter,71.0,53.0,NaN,NaN,brian gilbert,"betty mahmoody,william hoffer",43000000.0,metro-goldwyn-mayer,surround,NaN,13200000.0,9585.0,116,1991,"sally field, alfred molina, roshan seth",https://en.wikipedia.org/wiki/not_without_my_d...,"in 1984, an iranian physician, sayyed bozorg ""...",drama,english
4,rookie of the year,52.0,38.0,NaN,NaN,daniel stern,sam harper,56500000.0,20th century fox,NaN,NaN,10000000.0,21845.0,103,1993,"thomas ian nicholas, gary busey, dan hedaya",https://en.wikipedia.org/wiki/rookie_of_the_ye...,"henry rowengartner (nicholas), 12-year-old lit...","comedy, kids & family",english


In [2]:
data.columns

Index(['title', 'audience_score', 'tomato_meter', 'rating', 'rating_contents',
       'director', 'writer', 'box_office', 'distributor', 'sound_mix',
       'belongs_to_collection', 'budget', 'id', 'runtime', 'release_year',
       'cast', 'wiki_page', 'plot', 'genre', 'language'],
      dtype='object')

In [3]:
# Combine Director and Writer a single column with comma-separated values
data['Team'] = data.apply(lambda row: ', '.join(filter(None, [str(row['director']), str(row['writer'])])), axis=1)

In [4]:
data.drop(columns=['audience_score', 'tomato_meter','wiki_page', 'director', 'writer'], inplace=True)
data.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,plot,genre,language,Team
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979,"peter falk, alan arkin",the daughter of mild-mannered manhattan dentis...,comedy,english,"arthur hiller, andrew bergman"
1,race the sun,NaN,NaN,1700000.0,NaN,surround,NaN,0.0,55731.0,100,1996,"halle berry, james belushi","a new science teacher, miss sandra beecher, (h...",drama,english,"charles t. kanganis, nan"
2,paul blart: mall cop,pg,"mild crude humor, language, some violence",183000000.0,"sony pictures releasing,",NaN,NaN,26000000.0,NaN,91,2009,"jayma mays, keir o'donnell, bobby cannavale, s...","paul blart lives in west orange, new jersey wi...",comedy,english,"steve carr, kevin james"
3,not without my daughter,NaN,NaN,43000000.0,metro-goldwyn-mayer,surround,NaN,13200000.0,9585.0,116,1991,"sally field, alfred molina, roshan seth","in 1984, an iranian physician, sayyed bozorg ""...",drama,english,"brian gilbert, betty mahmoody,william hoffer"
4,rookie of the year,NaN,NaN,56500000.0,20th century fox,NaN,NaN,10000000.0,21845.0,103,1993,"thomas ian nicholas, gary busey, dan hedaya","henry rowengartner (nicholas), 12-year-old lit...","comedy, kids & family",english,"daniel stern, sam harper"


In [5]:
data['cast'] = (
    data['cast']
    .str.replace(";", ",", regex=False)
    .str.replace("and", ",", regex=False)
    .str.replace("with", ",", regex=False)
    .str.replace("voices of ", ",", regex=False)
    .str.replace("\r\n", ",", regex=False)
    .str.replace(r"\(.*?\)", "", regex=True)  # Remove text within parentheses
    .str.replace("Cast:", "", regex=False)        # Remove "Cast:"
)

data['Team'] = (
    data['Team']
    .str.replace(";", ",", regex=False)
    .str.replace("and", ",", regex=False)
    .str.replace("with", ",", regex=False)
    .str.replace("voices of ", ",", regex=False)
    .str.replace("\r\n", ",", regex=False)
    .str.replace(r"\(.*?\)", "", regex=True)  # Remove text within parentheses
    .str.replace("Director :", "", regex=False)  # Remove "Director :"
)
data.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,plot,genre,language,Team
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979,"peter falk, alan arkin",the daughter of mild-mannered manhattan dentis...,comedy,english,"arthur hiller, ,rew bergman"
1,race the sun,NaN,NaN,1700000.0,NaN,surround,NaN,0.0,55731.0,100,1996,"halle berry, james belushi","a new science teacher, miss sandra beecher, (h...",drama,english,"charles t. kanganis, nan"
2,paul blart: mall cop,pg,"mild crude humor, language, some violence",183000000.0,"sony pictures releasing,",NaN,NaN,26000000.0,NaN,91,2009,"jayma mays, keir o'donnell, bobby cannavale, s...","paul blart lives in west orange, new jersey wi...",comedy,english,"steve carr, kevin james"
3,not without my daughter,NaN,NaN,43000000.0,metro-goldwyn-mayer,surround,NaN,13200000.0,9585.0,116,1991,"sally field, alfred molina, roshan seth","in 1984, an iranian physician, sayyed bozorg ""...",drama,english,"brian gilbert, betty mahmoody,william hoffer"
4,rookie of the year,NaN,NaN,56500000.0,20th century fox,NaN,NaN,10000000.0,21845.0,103,1993,"thomas ian nicholas, gary busey, dan hedaya","henry rowengartner (nicholas), 12-year-old lit...","comedy, kids & family",english,"daniel stern, sam harper"


In [6]:
data['Team'] = data['Team'].fillna('').str.split(r',\s*')
data['cast'] = data['cast'].fillna('').str.split(r',\s*')
data.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,plot,genre,language,Team
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979,"[peter falk, alan arkin]",the daughter of mild-mannered manhattan dentis...,comedy,english,"[arthur hiller, , rew bergman]"
1,race the sun,NaN,NaN,1700000.0,NaN,surround,NaN,0.0,55731.0,100,1996,"[halle berry, james belushi]","a new science teacher, miss sandra beecher, (h...",drama,english,"[charles t. kanganis, nan]"
2,paul blart: mall cop,pg,"mild crude humor, language, some violence",183000000.0,"sony pictures releasing,",NaN,NaN,26000000.0,NaN,91,2009,"[jayma mays, keir o'donnell, bobby cannavale, ...","paul blart lives in west orange, new jersey wi...",comedy,english,"[steve carr, kevin james]"
3,not without my daughter,NaN,NaN,43000000.0,metro-goldwyn-mayer,surround,NaN,13200000.0,9585.0,116,1991,"[sally field, alfred molina, roshan seth]","in 1984, an iranian physician, sayyed bozorg ""...",drama,english,"[brian gilbert, betty mahmoody, william hoffer]"
4,rookie of the year,NaN,NaN,56500000.0,20th century fox,NaN,NaN,10000000.0,21845.0,103,1993,"[thomas ian nicholas, gary busey, dan hedaya]","henry rowengartner (nicholas), 12-year-old lit...","comedy, kids & family",english,"[daniel stern, sam harper]"


## Try Latent Factor Extraction

In [ ]:
# Import MultiLabelBinarizer
#from sklearn.preprocessing import MultiLabelBinarizer

# Create a one-hot encoded matrix from the list of names
#mlb = MultiLabelBinarizer()

#name_matrix = mlb.fit_transform(data2['names'])

#name_df = pd.DataFrame(name_matrix, columns=mlb.classes_)

#actor_counts = name_df.sum(axis=0)

# Filter out actors that appear only once (i.e., count == 1)
#filtered_name_df = name_df.loc[:, actor_counts > 10]

#print("One-hot encoded matrix:")
#print(name_df)


In [6]:
#fmatrix = filtered_name_df.values

# Alternatively, using .to_numpy()
#fmatrix = filtered_name_df.to_numpy()

In [ ]:
#import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

# Use a higher number of components to inspect explained variance
#n_components_initial = 2500
#svd = TruncatedSVD(n_components=n_components_initial, random_state=42)

#svd.fit_transform(name_matrix)

#explained_variance = svd.explained_variance_ratio_
#cumulative_variance = explained_variance.cumsum()

# Plot the cumulative explained variance (Scree Plot)
#plt.figure(figsize=(8, 4))
#plt.plot(range(1, n_components_initial + 1), cumulative_variance, marker='o')
#plt.xlabel('Number of Components')
#plt.ylabel('Cumulative Explained Variance')
#plt.title('Scree Plot for TSVD Components')
#plt.axhline(y=0.8, color='r', linestyle='--', label='80% Variance Threshold')
#plt.legend()
#plt.show()


In [81]:
#movies = name_df.apply(lambda row: list(name_df.columns[row == 1]), axis=1).tolist()

#exclusion_tokens = {" ", "Jr.", "on", "er", "ler", '', 'Am', 'S', 'eras', 'a peet'
#}

# 
#   cleaned_movies = [
#    [actor for actor in movie if actor not in exclusion_tokens]
#    for movie in movies
#]


In [ ]:
#from gensim import corpora, models

# Create a dictionary mapping actor names to IDs
#dictionary = corpora.Dictionary(cleaned_movies)

# Convert each movie into a bag-of-actors representation
#corpus = [dictionary.doc2bow(movie) for movie in movies]

In [ ]:
#num_topics = 10 
#lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42)

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis

# Enable notebook display for pyLDAvis
#pyLDAvis.enable_notebook()

# Prepare the visualization data
#vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization inline in your notebook
#vis_data


In [ ]:
#import pandas as pd
#import numpy as np
#from scipy.sparse import csr_matrix
#from sklearn.decomposition import TruncatedSVD
#import matplotlib.pyplot as plt

# Assume name_df is already defined and is a DataFrame with shape (2958, 4823)
# For example, you might have:
# name_df = pd.read_csv('your_file.csv')

# Convert the DataFrame to a SciPy sparse matrix (CSR format)
#sparse_matrix = csr_matrix(name_df.values)

# Define a range of SVD components to test (e.g., from 10 to 200 in steps of 10)
#component_range = range(100, 1000, 100)
#cumulative_explained = []

# Iterate over the range of components and compute cumulative explained variance
#for n_components in component_range:
#    svd = TruncatedSVD(n_components=n_components, random_state=42)
#    svd.fit(sparse_matrix)
#    explained = np.sum(svd.explained_variance_ratio_)
#    cumulative_explained.append(explained)
#   print(f"n_components: {n_components}, Cumulative Explained Variance: {explained:.4f}")

# Plot cumulative explained variance vs. number of components
#plt.figure(figsize=(8, 4))
#plt.plot(list(component_range), cumulative_explained, marker='o')
#plt.xlabel('Number of Components')
#plt.ylabel('Cumulative Explained Variance')
#plt.title('Optimal Number of SVD Components')
#plt.grid(True)
#plt.show()

In [ ]:
#import pandas as pd
#from scipy.sparse import csr_matrix
#from sklearn.decomposition import TruncatedSVD
#from sklearn.cluster import KMeans
#from sklearn.preprocessing import normalize
#from sklearn.metrics import silhouette_score
#import matplotlib.pyplot as plt

#svd = TruncatedSVD(n_components=400, random_state=42)
#movie_features = svd.fit_transform(sparse_matrix)

# Normalize the feature vectors
#movie_features = normalize(movie_features)

# Define a range of cluster numbers to try
#cluster_range = range(2, 25)  # trying cluster counts from 2 to 15
#silhouette_scores = []

#for n_clusters in cluster_range:
#    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#    cluster_labels = kmeans.fit_predict(movie_features)
#    score = silhouette_score(movie_features, cluster_labels)
#    silhouette_scores.append(score)
#    print(f"Number of clusters: {n_clusters} - Silhouette Score: {score:.4f}")

# Plot silhouette scores to visually inspect the optimal cluster count
#plt.figure(figsize=(8, 4))
#plt.plot(list(cluster_range), silhouette_scores, marker='o')
#plt.title("Silhouette Score vs. Number of Clusters")
#plt.xlabel("Number of Clusters")
#plt.ylabel("Silhouette Score")
#plt.xticks(list(cluster_range))
#plt.grid(True)
#plt.show()

In [ ]:
#sparse_matrix = csr_matrix(name_df.values)

# Define a range of SVD components to test
#component_options = [50, 100, 200, 400,500,600, 700, 800, 900]
#sil_scores = {}

#for n_components in component_options:
#    svd = TruncatedSVD(n_components=n_components, random_state=42)
#    features = svd.fit_transform(sparse_matrix)
#    features = normalize(features)
    
    # Use a fixed number of clusters; you can adjust based on your earlier analysis
#    kmeans = KMeans(n_clusters=10, random_state=42)
    labels = kmeans.fit_predict(features)
    
 #   score = silhouette_score(features, labels)
 #   sil_scores[n_components] = score
 #   print(f"n_components: {n_components} - Silhouette Score: {score:.4f}")

# Plot the silhouette scores against the number of components
#plt.figure(figsize=(8, 4))
#plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker='o')
#plt.xlabel('Number of SVD Components')
#plt.ylabel('Silhouette Score')
#plt.title('Silhouette Score vs. SVD Components')
#plt.grid(True)
#plt.show()

## Reviews Mapped to Team Members (Cast, Director, Writer) 

In [ ]:
# Load in Review
reviews = pd.read_csv('Rotten/rotten_tomatoes_movie_reviews.csv')
reviews.head()

In [ ]:
# Load in Movies
movies = pd.read_csv('Rotten/rotten_tomatoes_movies.csv')
movies.head()

In [ ]:
# Merge movies and reviews by 'id'
merged_df = pd.merge(movies, reviews, on='id', how='inner')
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
# Get unique names from the TEAM OR CAST column
#unique_names = pd.Series([name for sublist in data['cast'].apply(lambda x: list(set(x))).tolist() for name in sublist]).unique()
#unique_names = unique_names[unique_names != '']

unique_names = pd.Series([name for sublist in data['Team'].apply(lambda x: list(set(x))).tolist() for name in sublist]).unique()
unique_names = unique_names[unique_names != '']

unique_names.shape

In [ ]:
# Drop duplicates and clean names
unique_names = pd.Series(unique_names).drop_duplicates().str.strip()
sorted_unique_names = sorted([name for name in unique_names if name and len(name.split()) >= 2 and len(name) > 3])
len(sorted_unique_names)

In [ ]:
sorted_unique_names2 = [name for name in sorted_unique_names if len(name.split()[0]) != 1 and len(name.split()[-1]) > 1]
len(sorted_unique_names2)

In [ ]:
final_names = [name for name in sorted_unique_names2 if len(name.split()) < 4]
len(final_names)

In [ ]:
### DONT RUN THIS CELL

# Create a list to store the reviews for each actor
reviews = []

# Compile regex pattern once for efficiency
name_patterns = {name: re.compile(r'\b' + re.escape(name) + r'\b', re.IGNORECASE) for name in final_names}

# Iterate through each review in merged_df
for index, row in merged_df.iterrows():
    print(f"Processing row {index} ({(index / 1469543) * 100:.2f}%)")
    review_text = row['reviewText']
    if pd.isna(review_text):
        continue
    seen_review_ids = set()
    # Check each name pattern against the review text
    for name, pattern in name_patterns.items():
        if pattern.search(review_text) and row['reviewId'] not in seen_review_ids:
            reviews.append({
                'Name': name, 
                'Review': review_text, 
                'reviewId': row['reviewId'], 
                'creationDate': row['creationDate'],
                'audienceScore': row['audienceScore'],
                'tomatoMeter': row['tomatoMeter']
            })
            seen_review_ids.add(row['reviewId'])

# Convert the list to a DataFrame
#actor_reviews_df = pd.DataFrame(reviews)
team_reviews_df = pd.DataFrame(reviews)
#actor_reviews_df.head()
team_reviews_df.head()

In [ ]:
#actor_reviews_df.sort_values('Name')
team_reviews_df.sort_values('Name')

In [ ]:
#actor_reviews_df['Name'].nunique()
team_reviews_df['Name'].nunique()

In [128]:
# Save the DataFrame to a CSV file
#actor_reviews_df.to_csv('actor_reviews.csv', index=False)
team_reviews_df.to_csv('team_reviews.csv', index=False)

In [17]:
# Save the DataFrame to a JSON file
#import json

#with open('actor_reviews.json', 'w') as f:
#    json.dump(actor_reviews, f, indent=4)

In [7]:
# load in Reviews
dat = pd.read_csv('actor_reviews.csv')
team = pd.read_csv('team_reviews.csv')

In [8]:
dat1 = dat.drop_duplicates()
team1 = team.drop_duplicates()

In [ ]:
# Combine dat1 and team1
#combined_dat = pd.concat([dat1, team1], ignore_index=True)

In [ ]:
#combined_dat = combined_dat.drop_duplicates()

In [9]:
### Calculate Sentiment Scores for each Actor Review
analyzer = SentimentIntensityAnalyzer()

dat1['Sentiment'] = dat1['Review'].apply(lambda review: analyzer.polarity_scores(review))
team1['Sentiment'] = team1['Review'].apply(lambda review: analyzer.polarity_scores(review))

/var/folders/6_/6d4wvl895t3_bmq4xhpd5djc0000gp/T/ipykernel_18261/4281467891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat1['Sentiment'] = dat1['Review'].apply(lambda review: analyzer.polarity_scores(review))
/var/folders/6_/6d4wvl895t3_bmq4xhpd5djc0000gp/T/ipykernel_18261/4281467891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team1['Sentiment'] = team1['Review'].apply(lambda review: analyzer.polarity_scores(review))


In [10]:
# Extract Main Score 
dat1['compound'] = dat1['Sentiment'].apply(lambda sentiment: sentiment['compound'])
dat1 = dat1.sort_values('Name').reset_index(drop=True)

team1['compound'] = team1['Sentiment'].apply(lambda sentiment: sentiment['compound'])
team1 = team1.sort_values('Name').reset_index(drop=True)

/var/folders/6_/6d4wvl895t3_bmq4xhpd5djc0000gp/T/ipykernel_18261/746651818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat1['compound'] = dat1['Sentiment'].apply(lambda sentiment: sentiment['compound'])
/var/folders/6_/6d4wvl895t3_bmq4xhpd5djc0000gp/T/ipykernel_18261/746651818.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team1['compound'] = team1['Sentiment'].apply(lambda sentiment: sentiment['compound'])


In [11]:
dat1['creationDate'] = pd.to_datetime(dat1['creationDate'])

In [12]:
data.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,plot,genre,language,Team
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979,"[peter falk, alan arkin]",the daughter of mild-mannered manhattan dentis...,comedy,english,"[arthur hiller, , rew bergman]"
1,race the sun,NaN,NaN,1700000.0,NaN,surround,NaN,0.0,55731.0,100,1996,"[halle berry, james belushi]","a new science teacher, miss sandra beecher, (h...",drama,english,"[charles t. kanganis, nan]"
2,paul blart: mall cop,pg,"mild crude humor, language, some violence",183000000.0,"sony pictures releasing,",NaN,NaN,26000000.0,NaN,91,2009,"[jayma mays, keir o'donnell, bobby cannavale, ...","paul blart lives in west orange, new jersey wi...",comedy,english,"[steve carr, kevin james]"
3,not without my daughter,NaN,NaN,43000000.0,metro-goldwyn-mayer,surround,NaN,13200000.0,9585.0,116,1991,"[sally field, alfred molina, roshan seth]","in 1984, an iranian physician, sayyed bozorg ""...",drama,english,"[brian gilbert, betty mahmoody, william hoffer]"
4,rookie of the year,NaN,NaN,56500000.0,20th century fox,NaN,NaN,10000000.0,21845.0,103,1993,"[thomas ian nicholas, gary busey, dan hedaya]","henry rowengartner (nicholas), 12-year-old lit...","comedy, kids & family",english,"[daniel stern, sam harper]"


In [13]:
data['release_year'] = pd.to_datetime(data['release_year'].astype(str) + '-01-01')

In [14]:
## Expand Data by Cast Members
movie_exploded = data.explode(['cast'])
movie_exploded = movie_exploded.explode('Team')

In [15]:
movie_exploded.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,release_year,cast,plot,genre,language,Team
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979-01-01,peter falk,the daughter of mild-mannered manhattan dentis...,comedy,english,arthur hiller
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979-01-01,peter falk,the daughter of mild-mannered manhattan dentis...,comedy,english,
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979-01-01,peter falk,the daughter of mild-mannered manhattan dentis...,comedy,english,rew bergman
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979-01-01,alan arkin,the daughter of mild-mannered manhattan dentis...,comedy,english,arthur hiller
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,1979-01-01,alan arkin,the daughter of mild-mannered manhattan dentis...,comedy,english,


In [16]:
# Check Title Volume
movie_exploded['title'].nunique()

5298

In [17]:
movie_exploded.shape

(61265, 16)

In [18]:
# Merge with Final Data
total = movie_exploded.merge(dat1,left_on='cast', right_on='Name', how='left')

In [19]:
total.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,...,language,Team,Name,Review,reviewId,creationDate,audienceScore,tomatoMeter,Sentiment,compound
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,Good-natured but only memorable as a platform ...,1434746.0,2005-09-13,51.0,45.0,"{'neg': 0.0, 'neu': 0.845, 'pos': 0.155, 'comp...",0.2960
1,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,"Walter Hill continues doing what he does best,...",759469.0,2002-08-24,46.0,48.0,"{'neg': 0.125, 'neu': 0.717, 'pos': 0.158, 'co...",0.2263
2,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,A low-key comedy that features a wonderful per...,102781012.0,2023-02-17,0.0,NaN,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp...",0.7351
3,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,How brilliant is Wings of Desire? Understand t...,1993631.0,2011-06-25,93.0,98.0,"{'neg': 0.0, 'neu': 0.771, 'pos': 0.229, 'comp...",0.8442
4,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,Will somebody please offer Peter Falk a decent...,800485.0,2002-10-21,39.0,7.0,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",0.3182


In [20]:
total.rename(columns={
    'audienceScore': 'Cast_audienceScore',
    'tomatoMeter': 'Cast_tomatoMeter',
    'Sentiment': 'Cast_Sentiment',
    'compound': 'Cast_compound',
    'Review': 'Cast_Review',
    'reviewId': 'Cast_reviewId',
    'creationDate': 'Cast_creationDate'
}, inplace=True)
total.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,...,language,Team,Name,Cast_Review,Cast_reviewId,Cast_creationDate,Cast_audienceScore,Cast_tomatoMeter,Cast_Sentiment,Cast_compound
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,Good-natured but only memorable as a platform ...,1434746.0,2005-09-13,51.0,45.0,"{'neg': 0.0, 'neu': 0.845, 'pos': 0.155, 'comp...",0.2960
1,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,"Walter Hill continues doing what he does best,...",759469.0,2002-08-24,46.0,48.0,"{'neg': 0.125, 'neu': 0.717, 'pos': 0.158, 'co...",0.2263
2,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,A low-key comedy that features a wonderful per...,102781012.0,2023-02-17,0.0,NaN,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp...",0.7351
3,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,How brilliant is Wings of Desire? Understand t...,1993631.0,2011-06-25,93.0,98.0,"{'neg': 0.0, 'neu': 0.771, 'pos': 0.229, 'comp...",0.8442
4,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,english,arthur hiller,peter falk,Will somebody please offer Peter Falk a decent...,800485.0,2002-10-21,39.0,7.0,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",0.3182


In [21]:
# Check Title Overlap
total['title'].nunique()

5298

In [22]:
total.shape

(2427263, 24)

In [23]:
# Drop duplicates in the 'total' DataFrame
total.drop(columns=['Cast_Sentiment'], inplace=True)

In [24]:
# Drop duplicates in the 'total' DataFrame
total.drop_duplicates(inplace=True)

In [25]:
team1_indexed = team1.set_index('Name')

# Merge using the index on the right, which can be faster
total_team = total.merge(team1_indexed, left_on='Team', right_index=True, how='left')

In [26]:
total_team.rename(columns={
    'Review': 'Team_Review',
    'reviewId': 'Team_reviewId',
    'creationDate': 'Team_creationDate',
    'audienceScore': 'Team_audienceScore',
    'tomatoMeter': 'Team_tomatoMeter'
}, inplace=True)
total_team.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,...,Cast_audienceScore,Cast_tomatoMeter,Cast_compound,Team_Review,Team_reviewId,Team_creationDate,Team_audienceScore,Team_tomatoMeter,Sentiment,compound
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,51.0,45.0,0.296,"Love Story (1970) is a sentimental, romantic t...",11709.0,2000-01-01,75.0,66.0,"{'neg': 0.106, 'neu': 0.567, 'pos': 0.326, 'co...",0.7351
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,51.0,45.0,0.296,Another director (not Arthur Hiller) would hav...,1920466.0,2010-07-27,82.0,93.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,51.0,45.0,0.296,"It's also unexpectedly likeable, thanks to the...",22883.0,2000-01-01,52.0,33.0,"{'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'comp...",0.8442
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,51.0,45.0,0.296,Arthur Hiller proves he is the wrong director ...,1978539.0,2011-04-18,29.0,NaN,"{'neg': 0.176, 'neu': 0.824, 'pos': 0.0, 'comp...",-0.7351
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,51.0,45.0,0.296,Arthur Hiller directed this muddied comic stud...,102768098.0,2023-01-11,43.0,30.0,"{'neg': 0.065, 'neu': 0.758, 'pos': 0.177, 'co...",0.6124


In [27]:
# How many don't have any reviews
num_na_team_creation_date = total_team[total_team['Team_creationDate'].isna()]['title'].nunique()
num_na_cast_creation_date = total_team[total_team['Cast_creationDate'].isna()]['title'].nunique()
num_na_both_creation_date = total_team[total_team['Team_creationDate'].isna() & total_team['Cast_creationDate'].isna()]['title'].nunique()
num_total_titles = total_team['title'].nunique()

percentage_na_team_creation_date = (num_na_team_creation_date / num_total_titles) * 100
percentage_na_cast_creation_date = (num_na_cast_creation_date / num_total_titles) * 100
percentage_na_both_creation_date = (num_na_both_creation_date / num_total_titles) * 100

print(f"Percentage of titles with Team_creationDate NA: {percentage_na_team_creation_date}%")
print(f"Percentage of titles with Cast_creationDate NA: {percentage_na_cast_creation_date}%")
print(f"Percentage of titles with both Team_creationDate and Cast_creationDate NA: {percentage_na_both_creation_date}%")

Percentage of titles with Team_creationDate NA: 49.565873914684786%
Percentage of titles with Cast_creationDate NA: 62.533031332578325%
Percentage of titles with both Team_creationDate and Cast_creationDate NA: 33.76745941864855%


In [28]:
total_team.shape

(105372512, 30)

In [30]:
total_team.drop(columns=['Sentiment'], inplace=True)
total_team.rename(columns={'compound': 'Team_compound'}, inplace=True)
total_team.head()

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,...,Cast_creationDate,Cast_audienceScore,Cast_tomatoMeter,Cast_compound,Team_Review,Team_reviewId,Team_creationDate,Team_audienceScore,Team_tomatoMeter,Team_compound
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.296,"Love Story (1970) is a sentimental, romantic t...",11709.0,2000-01-01,75.0,66.0,0.7351
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.296,Another director (not Arthur Hiller) would hav...,1920466.0,2010-07-27,82.0,93.0,0.0000
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.296,"It's also unexpectedly likeable, thanks to the...",22883.0,2000-01-01,52.0,33.0,0.8442
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.296,Arthur Hiller proves he is the wrong director ...,1978539.0,2011-04-18,29.0,NaN,-0.7351
0,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.296,Arthur Hiller directed this muddied comic stud...,102768098.0,2023-01-11,43.0,30.0,0.6124


In [31]:
# Fitler for Point in Time Data, what to do with NA values?
total_team_filter = total_team[(total_team['Team_creationDate'] < total_team['release_year']) | (total_team['Team_creationDate'].isna())]

In [32]:
total_team_filter

,title,rating,rating_contents,box_office,distributor,sound_mix,belongs_to_collection,budget,id,runtime,...,Cast_creationDate,Cast_audienceScore,Cast_tomatoMeter,Cast_compound,Team_Review,Team_reviewId,Team_creationDate,Team_audienceScore,Team_tomatoMeter,Team_compound
9,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2005-09-13,51.0,45.0,0.2960,NaN,NaN,NaN,NaN,NaN,NaN
10,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2002-08-24,46.0,48.0,0.2263,NaN,NaN,NaN,NaN,NaN,NaN
11,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2023-02-17,0.0,NaN,0.7351,NaN,NaN,NaN,NaN,NaN,NaN
12,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2011-06-25,93.0,98.0,0.8442,NaN,NaN,NaN,NaN,NaN,NaN
13,the in-laws,NaN,NaN,38200000.0,NaN,NaN,NaN,9000000.0,NaN,103,...,2002-10-21,39.0,7.0,0.3182,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427220,the octagon,NaN,NaN,19000000.0,american cinema releasing,NaN,NaN,4000000.0,24825.0,103,...,2015-04-25,79.0,100.0,-0.7923,NaN,NaN,NaN,NaN,NaN,NaN
2427221,the octagon,NaN,NaN,19000000.0,american cinema releasing,NaN,NaN,4000000.0,24825.0,103,...,2018-07-03,33.0,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN
2427222,the octagon,NaN,NaN,19000000.0,american cinema releasing,NaN,NaN,4000000.0,24825.0,103,...,2003-06-23,79.0,92.0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN
2427224,the octagon,NaN,NaN,19000000.0,american cinema releasing,NaN,NaN,4000000.0,24825.0,103,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# See drop in volume
total_team['title'].nunique() - total_team_filter['title'].nunique()

1473

In [ ]:
# Create Key for Aggregation
total1['movie_id'] = total1['title'] + '_' + total1['release_year'].dt.year.astype(str)
total1.head()

In [ ]:
# Group by 'movie_id', aggregate metrics and keep all other columns
aggregated_data = total1.groupby('movie_id').agg(
    Review_Score_sum=('compound', 'sum'),
    Review_Score_mean=('compound', 'mean'),
    Audience_Score_Mean=('audienceScore', 'mean'),
    Tomato_Meter_Mean=('tomatoMeter', 'mean'),
    #Review=('Review', lambda x: ' '.join(x)),
    Num_reviews=('reviewId', 'nunique'),
    **{col: (col, 'first') for col in total1.columns if col not in ['movie_id', 'compound', 'Review', 'reviewId']}
).reset_index()

aggregated_data.head()

In [ ]:
aggregated_data.shape

In [ ]:
aggregated_data.columns

In [ ]:
total1['movie_id'].nunique()

In [ ]:

# Sentiment Score Sum vs Box Office Revenue
plt.figure(figsize=(10, 6))
plt.scatter(aggregated_data['Review_Score_sum'], aggregated_data['box_office'], alpha=0.5)
plt.title('Review Sentiment Score Sum vs Box Office Numeric')
plt.xlabel('Review Sentiment Score Sum')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

# Sentiment Score Mean vs Box Office Revenue
plt.figure(figsize=(10, 6))
plt.scatter(aggregated_data['Review_Score_mean'], aggregated_data['box_office'], alpha=0.5)
plt.title('Review Sentiment Score Mean vs Box Office Numeric')
plt.xlabel('Review Sentiment Score Mean')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

In [ ]:
aggregated_data['Review_Score_sum'].describe()

In [ ]:
aggregated_data['Review_Score_mean'].describe()

In [ ]:
# Number of Reviews vs Box Office
plt.figure(figsize=(10, 6))
plt.scatter(aggregated_data['Num_reviews'], aggregated_data['box_office'], alpha=0.5)
plt.title('Number of Reviews vs Box Office')
plt.xlabel('Number of Reviews')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

In [ ]:
aggregated_data['Num_reviews'].describe()

In [ ]:
# Separate aggregated_data number of reviews into quantiles
aggregated_data['num_reviews_quantiles'] = pd.qcut(aggregated_data['Num_reviews'], q=5, labels=False)

# Check Interaction with Box Office
plt.figure(figsize=(10, 6))
aggregated_data.boxplot(column='box_office', by='num_reviews_quantiles')
plt.title('Box Office Numeric by Number of Reviews Quantiles')
plt.suptitle('')
plt.xlabel('Number of Reviews Quantiles')
plt.ylabel('Box Office Revenue')
plt.grid(True)
plt.show()

# Separate aggregated_data compound mean into quantiles
aggregated_data['compound_mean_quantiles'] = pd.qcut(aggregated_data['Review_Score_mean'], q=5, labels=False)

plt.figure(figsize=(10, 6))
aggregated_data.boxplot(column='box_office', by='compound_mean_quantiles')
plt.title('Box Office Numeric by Compound Mean Quantiles')
plt.suptitle('')
plt.xlabel('Compound Mean Quantiles')
plt.ylabel('Box Office Revenue')
plt.grid(True)
plt.show()


In [ ]:
### Movies that Lost Money
aggregated_data[aggregated_data['budget'] > aggregated_data['box_office']]

In [ ]:
### Cluster on these metrics
matched_metrics = aggregated_data[['Review_Score_sum', 'Review_Score_mean', 'Audience_Score_Mean', 'Tomato_Meter_Mean', 'Num_reviews']]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Ensure no NA values in matched_metrics
matched_metrics = matched_metrics.dropna()

# Normalize the variables
scaler = StandardScaler()
matched_metrics_scaled = scaler.fit_transform(matched_metrics)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(matched_metrics_scaled)

# Find the optimal number of clusters
inertia_values = []
silhouette_scores = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(principal_components)
    inertia_values.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(principal_components, kmeans.labels_))

# Plot inertia 
plt.figure(figsize=(10, 6))
plt.plot(K, inertia_values, marker='o')
plt.title('Inertia for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

# Plot silhouette scores 
plt.figure(figsize=(10, 6))
plt.plot(K, silhouette_scores, marker='x')
plt.title('Silhouette Score for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()

In [ ]:
# Perform KMeans clustering 
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(principal_components)
labels = kmeans.labels_

# Add the cluster labels to the matched_metrics 
matched_metrics['Cluster'] = labels

# Define cluster labels
cluster_labels = {3: 'Blockbuster Ensemble', 2: 'Critically Acclaimed', 0: 'Steady Performers', 1: 'Low Impact Ensemble'}
matched_metrics['Cluster Label'] = matched_metrics['Cluster'].map(cluster_labels)

# Plot clusters with labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=matched_metrics['Cluster'], cmap='viridis')
plt.title('Clusters of Matched Metrics')
plt.xlabel('First Metric')
plt.ylabel('Second Metric')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
# Add legend with cluster labels
handles, _ = scatter.legend_elements()
legend_labels = [cluster_labels[i] for i in range(len(handles))]
plt.legend(handles, legend_labels, title="Cluster Labels")
plt.show()

In [ ]:
# For each cluster, get description of principal components
cluster_descriptions = matched_metrics.groupby('Cluster').mean()
cluster_descriptions